### Coleta de dados

Este notebook tem por finalidade realizar a coleta de dados que será utilizada no estudo de hiperparâmetros

#### Dados utilizados

* Ações do Facebook (FB)
* Ações da Petrobras (PBR)
* Ações da Apple (AAPL)
* Ações da Tesla (TSLA)

Serão armazenadas apenas as variáveis de abertura e fechamento para os períodos de 1, 3 e 5 anos

In [4]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import numpy as np

In [2]:
ativos = ['FB', 'PBR', 'AAPL', 'TSLA']

datas_inicio = [dt.datetime(2016, 3, 1), dt.datetime(2018, 3, 1), dt.datetime(2020, 3, 1)]

data_fim = dt.datetime(2021, 3, 1)

In [7]:
end = dt.datetime(2020, 6, 1)
start = dt.datetime(2019, 1, 1)

df = web.DataReader("FB", 'yahoo', start, end)


df = df.reset_index()
df = df.drop(columns=['Date','High','Low','Volume','Adj Close'])

nomes = {'Close': 'Alvo',
         'Open': 'Abertura'}

df = df.rename(columns = nomes)

356


,Abertura,Alvo
0,128.990005,135.679993
1,134.690002,131.740005
2,134.009995,137.949997
3,137.559998,138.050003
4,139.889999,142.529999


In [ ]:
for at in ativos:
    
    for i, dt_i in enumerate(datas_inicio):
        
        if i == 0:
            periodo = 5
        elif i == 1:
            periodo = 3
        elif i == 2:
            periodo = 1
            
        df = web.DataReader(at, 'yahoo', dt_i, data_fim)
        
        df = df.reset_index()

        nomes = {'Close': 'Close 0',
                 'Open': 'Open 0'}

        df = df.rename(columns = nomes)

        df['Date2'] = df['Date'].astype(str).tolist()
        df['Ano'] = df['Date2'].str.split('-').str[0]
        df['Mes'] = df['Date2'].str.split('-').str[1]
        df['Dia'] = df['Date2'].str.split('-').str[2]

        dict_day = { 0: 'Segunda', 1: 'Terca', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta'}

        dia = []
        for wd in df['Date'].dt.dayofweek:
            dia.append(dict_day[wd])

        df['Dia_semana'] = dia

        for i in range(1, 6):
            df['Close '+str(i)] = df['Close '+str(i-1)].shift(1)
            df['Open '+str(i)] = df['Open '+str(i-1)].shift(1)

        df = df.drop(columns=['Date','Date2','High','Low','Volume','Adj Close'])

        df.rename(columns={'Close 0':'Target'}, inplace=True)
        df.dropna(inplace=True)
        
        df['Periodo'] = np.where(df['Dia'].astype(int)<8, 1, 
                        np.where(df['Dia'].astype(int)<15,2,
                        np.where(df['Dia'].astype(int)<21,3,4)))
        
        df['Anos'] = df['Ano']
        df['Meses'] = df['Mes']
        df['Dias'] = df['Dia']
        df['Dias_semana'] = df['Dia_semana']
        df['Periodos'] = df['Periodo']
        df = pd.get_dummies(df, columns=['Ano','Mes','Dia','Dia_semana','Periodo'])
        
        file = "Datasets/Acoes_2/"+str(at)+"_"+str(periodo)+'.csv'
        
        df.to_csv(file)
        print(file)

Datasets/Acoes_2/FB_5.csv
Datasets/Acoes_2/FB_3.csv
Datasets/Acoes_2/FB_1.csv
Datasets/Acoes_2/PBR_5.csv
Datasets/Acoes_2/PBR_3.csv
Datasets/Acoes_2/PBR_1.csv
